# **Pretrained Encyclopedia: Weakly Supervised Knowledge-Pretrained Language Model**

**Authors:** Wenhan Xiong, Jingfei Du, William Yang Wang, Veselin Stoyanov

## **Import Modules**

In [279]:
!pip install transformers

In [280]:
import transformers

In [281]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [282]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

import tokenizers


In [283]:
import torch
from transformers import BertForQuestionAnswering

bert_qa_model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
from transformers import BertTokenizer

bert_qa_tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [284]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
import textwrap

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [285]:
from difflib import SequenceMatcher

In [286]:
from transformers import pipeline

## **Dataset**

In [287]:
squad_train_data = pd.read_json('https://raw.githubusercontent.com/Abhiramnaidu1/ML_dataset/main/squad/train-v2.0.json')
squad_test_data = pd.read_json('https://raw.githubusercontent.com/Abhiramnaidu1/ML_dataset/main/squad/dev-v2.0.json')

In [288]:
squad_train_data.head(1)['data']

0    {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
Name: data, dtype: object

In [289]:
titles = []
contexts = []
questions = []
ids =[]
answers = []
answer_start = []
titles_test = []
contexts_test = []
questions_test = []
ids_test =[]
answers_test = []
answer_start_test = []
is_impossible = []
is_impossible_test = []
for i in squad_train_data['data']:
  title = i['title']
  for j in i['paragraphs']:
    context = j['context']
    for k in j['qas']:
      question = k['question']
      id_ = k['id']
      is_impossible_ = k['is_impossible']
      for l in k['answers']:
        titles.append(title)
        contexts.append(context)
        questions.append(question)
        ids.append(id_)
        answers.append(l['text'])
        answer_start.append(l['answer_start'])
        is_impossible.append(is_impossible_)

for i in squad_test_data['data']:
  title_test = i['title']
  for j in i['paragraphs']:
    context_test = j['context']
    for k in j['qas']:
      question_test = k['question']
      id_test_ = k['id']
      is_impossible_test_ = k['is_impossible']
      for l in k['answers']:
        titles_test.append(title_test)
        contexts_test.append(context_test)
        questions_test.append(question_test)
        ids_test.append(id_test_)
        answers_test.append(l['text'])
        answer_start_test.append(l['answer_start'])
        is_impossible_test.append(is_impossible_test_)

In [290]:
'''for i in is_impossible:
  if i == 'True':
    print('h')'''
for i in is_impossible_test:
  if i == 'True':
    print('h')

In [291]:
train = pd.DataFrame({'title':titles, 'question':questions, 'id':ids, 'answers':answers, 'answer_start':answer_start ,'context':contexts})
test = pd.DataFrame({'title':titles_test, 'question':questions_test, 'id':ids_test, 'answers':answers_test, 'answer_start':answer_start_test ,'context':contexts_test})

In [292]:
train.head(5)

,title,question,id,answers,answer_start,context
0,Beyoncé,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,Beyoncé,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,Beyoncé,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,Beyoncé,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,Beyoncé,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...


## **Training Model**

Taking 100 rows of data due to memory constraits

In [293]:
train= train.head(100)
MAX_LEN=135

In [294]:
rec = train.shape[0]  # Number of records in the training set
inputs = np.ones((rec, MAX_LEN), dtype = 'int32') # Input vector
attention_mask = np.zeros((rec, MAX_LEN), dtype = 'int32') # Attention Mask
token_type_ids = np.zeros((rec, MAX_LEN), dtype = 'int32') # Tokens produced
start_tokens = np.zeros((rec, MAX_LEN), dtype = 'int32') # Start logit for answer
end_tokens = np.zeros((rec, MAX_LEN), dtype = 'int32') # End logit for answer

for i in range(rec):

  context = ' '+' '.join(train.loc[i, 'context'].split())
  answer = ' '+' '.join(train.loc[i, 'answers'].split())
  question = ' '+' '.join(train.loc[i, 'question'].split())

  start_idx = train.loc[i, 'answer_start']

  chars = np.zeros((len(context)))
  chars[start_idx:start_idx + len(answer)] = 1
  if context[start_idx - 1] == ' ':
    chars[start_idx - 1] = 1
  
  enc1 = bert_qa_tokenizer.encode(context)
  enc2 = bert_qa_tokenizer.encode(question)
  print(enc1)
  # For resource limitations only.

  if len(enc1) + len(enc2) + 4 < MAX_LEN:

    #creating offsets
    offsets = []
    start_idx = 0

    for t in enc1:
      w = bert_qa_tokenizer.decode([t])
      offsets.append((start_idx, start_idx + len(w)))
      start_idx += len(w)
    
    # Those which are a part of the answer

    tokens = []
    for j, (a, b) in enumerate(offsets):
      sum_ = np.sum(chars[a:b])
      if sum_ > 0:
        tokens.append(j)

    # The input for roberta is in the form <s> Question </s></s> Context </s>
    
    inputs[i, :len(enc1) + len(enc2) + 4] = [0] + enc2 + [2,2] + enc1 + [2]

    attention_mask[i, :len(enc1) + len(enc2) + 4] = 1

    if len(tokens) > 0:
      start_tokens[i, tokens[0] + 1] = 1
      end_tokens[i, tokens[-1] + 1] = 1

[101, 20773, 21025, 19358, 22815, 1011, 5708, 1006, 1013, 12170, 23432, 29715, 3501, 29678, 12325, 29685, 1013, 10506, 1011, 10930, 2078, 1011, 2360, 1007, 1006, 2141, 2244, 1018, 1010, 3261, 1007, 2003, 2019, 2137, 3220, 1010, 6009, 1010, 2501, 3135, 1998, 3883, 1012, 2141, 1998, 2992, 1999, 5395, 1010, 3146, 1010, 2016, 2864, 1999, 2536, 4823, 1998, 5613, 6479, 2004, 1037, 2775, 1010, 1998, 3123, 2000, 4476, 1999, 1996, 2397, 4134, 2004, 2599, 3220, 1997, 1054, 1004, 1038, 2611, 1011, 2177, 10461, 1005, 1055, 2775, 1012, 3266, 2011, 2014, 2269, 1010, 25436, 22815, 1010, 1996, 2177, 2150, 2028, 1997, 1996, 2088, 1005, 1055, 2190, 1011, 4855, 2611, 2967, 1997, 2035, 2051, 1012, 2037, 14221, 2387, 1996, 2713, 1997, 20773, 1005, 1055, 2834, 2201, 1010, 20754, 1999, 2293, 1006, 2494, 1007, 1010, 2029, 2511, 2014, 2004, 1037, 3948, 3063, 4969, 1010, 3687, 2274, 8922, 2982, 1998, 2956, 1996, 4908, 2980, 2531, 2193, 1011, 2028, 3895, 1000, 4689, 1999, 2293, 1000, 1998, 1000, 3336, 2879, 1000

In [295]:
def build_model():
  ids = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)
  att = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)
  tok = tf.keras.layers.Input((MAX_LEN,), dtype = tf.int32)
  print(att)

  
  # x=DistlBert_model

  
  
  bert_model = bert_qa_model
 
  x = bert_model(ids, attention_mask=att, token_type_ids = tok)
  print(att)
  # For start logit

  x1 = tf.keras.layers.Dropout(0.1)(x[0])
  x1 = tf.keras.layers.Conv1D(1,1)(x1)
  x1 = tf.keras.layers.Flatten()(x1)
  x1 = tf.keras.layers.Activation('softmax')(x1)
  
  # For end logit

  x2 = tf.keras.layers.Dropout(0.1)(x[0]) 
  x2 = tf.keras.layers.Conv1D(1,1)(x2)
  x2 = tf.keras.layers.Flatten()(x2)
  x2 = tf.keras.layers.Activation('softmax')(x2)

  # Initalising the model

  model = tf.keras.models.Model(inputs = [ids, att, tok], outputs = [x1, x2])
  optimizer = tf.keras.optimizers.Adam(learning_rate = 3e-5)
  model.compile(loss='categorical_crossentropy', optimizer = optimizer)

  return model

## **Model Evaluation**

In [296]:
test = test.drop_duplicates( subset=None, keep="first", inplace=False)

### **BERT Question answering model finetuned with SQuaD dataset**

In [297]:
def bert_qa_answer_question(question, answer_text):
    
    # Apply the tokenizer and segment input text of question and context pair.
    input_ids = bert_qa_tokenizer.encode(question, answer_text)
    sep_index = input_ids.index(bert_qa_tokenizer.sep_token_id)
    num_seg_a = sep_index + 1
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    assert len(segment_ids) == len(input_ids)
    outputs = bert_qa_model(torch.tensor([input_ids]), 
                    token_type_ids=torch.tensor([segment_ids]), 
                    return_dict=True) 

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)

    # Get the string versions of the input tokens.
    tokens = bert_qa_tokenizer.convert_ids_to_tokens(input_ids)
    answer = tokens[answer_start]

    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        else:
            answer += ' ' + tokens[i]

    print('Answer: "' + answer + '"')
    return answer

#### Model Verification

In [298]:


# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80) 

bert_abstract = "We introduce a new language representation model called BERT, which stands for Bidirectional Encoder Representations from Transformers. Unlike recent language representation models (Peters et al., 2018a; Radford et al., 2018), BERT is designed to pretrain deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context in all layers. As a result, the pre-trained BERT model can be finetuned with just one additional output layer to create state-of-the-art models for a wide range of tasks, such as question answering and language inference, without substantial taskspecific architecture modifications. BERT is conceptually simple and empirically powerful. It obtains new state-of-the-art results on eleven natural language processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1 question answering Test F1 to 93.2 (1.5 point absolute improvement) and SQuAD v2.0 Test F1 to 83.1 (5.1 point absolute improvement)."

print(wrapper.fill(bert_abstract))

We introduce a new language representation model called BERT, which stands for
Bidirectional Encoder Representations from Transformers. Unlike recent language
representation models (Peters et al., 2018a; Radford et al., 2018), BERT is
designed to pretrain deep bidirectional representations from unlabeled text by
jointly conditioning on both left and right context in all layers. As a result,
the pre-trained BERT model can be finetuned with just one additional output
layer to create state-of-the-art models for a wide range of tasks, such as
question answering and language inference, without substantial taskspecific
architecture modifications. BERT is conceptually simple and empirically
powerful. It obtains new state-of-the-art results on eleven natural language
processing tasks, including pushing the GLUE score to 80.5% (7.7% point absolute
improvement), MultiNLI accuracy to 86.7% (4.6% absolute improvement), SQuAD v1.1
question answering Test F1 to 93.2 (1.5 point absolute improvement) 

In [299]:
question = "What are some example applications of BERT?"

bert_qa_answer_question(question, bert_abstract)

Answer: "question answering and language inference"


'question answering and language inference'

In [300]:
question = "What does the 'B' in BERT stand for?"

bert_qa_answer_question(question, bert_abstract)

Answer: "bidirectional encoder representations from transformers"


'bidirectional encoder representations from transformers'

### **Distilled BERT base model finetuned with SQuaD dataset**




In [301]:
nlp_qa = pipeline('question-answering')

No model was supplied, defaulted to distilbert-base-cased-distilled-squad (https://huggingface.co/distilbert-base-cased-distilled-squad)


In [302]:
def distil_bert_answer_model(question,context):
  return nlp_qa(question,context)['answer']

#### Model Verification

In [303]:
question = "What does the 'B' in BERT stand for?"
distil_bert_answer_model(question,bert_abstract)

'Bidirectional Encoder Representations from Transformers'

In [304]:
question = "What are some example applications of BERT?"
nlp_qa(question,bert_abstract)['answer']

'question answering and language inference'

## **Evaluation**

In [305]:
test

,title,question,id,answers,answer_start,context
0,Normans,In what country is Normandy located?,56ddde6b9a695914005b9628,France,159,The Normans (Norman: Nourmands; French: Norman...
4,Normans,When were the Normans in Normandy?,56ddde6b9a695914005b9629,10th and 11th centuries,94,The Normans (Norman: Nourmands; French: Norman...
5,Normans,When were the Normans in Normandy?,56ddde6b9a695914005b9629,in the 10th and 11th centuries,87,The Normans (Norman: Nourmands; French: Norman...
8,Normans,From which countries did the Norse originate?,56ddde6b9a695914005b962a,"Denmark, Iceland and Norway",256,The Normans (Norman: Nourmands; French: Norman...
12,Normans,Who was the Norse leader?,56ddde6b9a695914005b962b,Rollo,308,The Normans (Norman: Nourmands; French: Norman...
...,...,...,...,...,...,...
20287,Force,What is a very seldom used unit of mass in the...,5737aafd1c456719005744fd,slug,274,"The pound-force has a metric counterpart, less..."
20288,Force,What is a very seldom used unit of mass in the...,5737aafd1c456719005744fd,metric slug,267,"The pound-force has a metric counterpart, less..."
20291,Force,What is a very seldom used unit of mass in the...,5737aafd1c456719005744fd,the metric slug,263,"The pound-force has a metric counterpart, less..."
20292,Force,What seldom used term of a unit of force equal...,5737aafd1c456719005744fe,kip,712,"The pound-force has a metric counterpart, less..."


In [306]:
test=test.head(500)

### **AnswerPrediction using BERT Question Answering Model**

In [307]:
test['predicted_answer_bert_qa'] = test.apply(lambda x:bert_qa_answer_question(x.question,x.context),axis=1)

Answer: "france"
Answer: "10th and 11th centuries"
Answer: "10th and 11th centuries"
Answer: "denmark , iceland and norway"
Answer: "rollo"
Answer: "10th"
Answer: "10th"
Answer: "10th"
Answer: "william the conqueror"
Answer: "richard i of normandy"
Answer: "christian"
Answer: "christian"
Answer: "norseman , viking"
Answer: "norseman , viking"
Answer: "9th century"
Answer: "911"
Answer: "king charles iii of west francia"
Answer: "river seine"
Answer: "river seine"
Answer: "rollo"
Answer: "norse religion and old norse language with catholicism ( christianity"
Answer: "north"
Answer: "north"
Answer: "fighting horsemen"
Answer: "the pechenegs , the bulgars , and especially the seljuk turks"
Answer: "the pechenegs , the bulgars , and especially the seljuk turks"
Answer: "the pechenegs , the bulgars , and especially the seljuk turks"
Answer: "1050s"
Answer: "1050s"
Answer: "1060s"
Answer: "1060s"
Answer: "alexius komnenos"
Answer: "afranji"
Answer: "oursel"
Answer: "turkish forces"
Answer: "

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### **AnswerPrediction using Distilled BERT Model**

In [308]:
test['predicted_answer_distil_bert_qa'] = test.apply(lambda x:distil_bert_answer_model(x.question,x.context),axis=1)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## **Metrics**

### **Test Normalization**

In [309]:

stop_words = stopwords.words('english')
ps=PorterStemmer()

In [310]:
def normalize_text(answer):

  def split_answer(answer):
    return answer.split(' ')
  
  def answer_wo_stopwords(answer):
    stop_words = stopwords.words('english')
    stop_word_list=[]
    for i in answer:
      if i not in stop_words:
        stop_word_list.append(i)
    return stop_word_list
      

  def root_answer(answer):
    ps=PorterStemmer()
    stem_list=[]
    for i in answer:
      stem_word= ps.stem(i)
      stem_list.append(stem_word)
    return stem_list
  
  def lower_answer(answer):
    new_answer = ''.join(answer)
    low_answer = new_answer.lower()
    return low_answer
      
  splitted_answer= split_answer(answer)
  stopwords_no_answer =answer_wo_stopwords(splitted_answer)
  rootwords_answer = root_answer(stopwords_no_answer)
  norm_answer = lower_answer(rootwords_answer)
  
  return norm_answer

In [311]:
normalize_text("in the late 1990s")


'late1990'

### **EM score**

In [312]:


def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def Exact_Match(word1,word2):
  norm_word1=normalize_text(word1)
  norm_word2=normalize_text(word2)
  EM_score = similar(norm_word1,norm_word2)
  return EM_score


In [313]:
test['EM_score_bert_qa'] = test.apply(lambda x:Exact_Match(x.answers,x.predicted_answer_bert_qa),axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [314]:
test['EM_score_distil_bert_qa'] = test.apply(lambda x:Exact_Match(x.answers,x.predicted_answer_distil_bert_qa),axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### **F1 Measure**

In [322]:

binarizer = MultiLabelBinarizer()

def f1_score_model(y_true,y_pred):
  binarizer.fit(y_true)

  f1_score_value=f1_score(binarizer.transform(y_true), binarizer.transform(y_pred), average='macro')
  return f1_score_value

## **Summary**

**EM Score**

In [318]:
EM_score_bert_qa = test['EM_score_bert_qa'].mean()
EM_score_distil_bert_qa= test['EM_score_distil_bert_qa'].mean()
print('Exact Match score for Bert Question answering model is %f',EM_score_bert_qa)
print('Exact Match score for Distiled Bert Pipeling model is %f',EM_score_distil_bert_qa)

Exact Match score for Bert Question answering model is %f 0.8325303277597107
Exact Match score for Distiled Bert Pipeling model is %f 0.7965695690122543


**F1 Measure**

In [323]:
F1_bert_qa = f1_score_model(test['answers'],test['predicted_answer_bert_qa'])
F1_distilbert_qa = f1_score_model(test['answers'],test['predicted_answer_distil_bert_qa'])
print('F1 score for Bert Question answering model is %f',F1_bert_qa)
print('F1 score for Distiled Bert Pipeling model is %f',F1_distilbert_qa)

F1 score for Bert Question answering model is %f 0.48674160330673
F1 score for Distiled Bert Pipeling model is %f 0.7696055688465991


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:876: UserWarning: unknown class(es) ['Π'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))


## **Conclusion**

## **References**

1.	PRETRAINED ENCYCLOPEDIA: WEAKLY SUPERVISED KNOWLEDGE-PRETRAINED LANGUAGE MODEL Wenhan Xiong, Jingfei Du, William Yang Wang, Veselin Stoyanov
2.	BERT — transformers 4.12.5 documentation (huggingface.co)
3.	[1905.07129] ERNIE: Enhanced Language Representation with Informative Entities (arxiv.org)
4.	https://arxiv.org/abs/1903.06164
5.	https://arxiv.org/abs/1406.3676
6.	https://arxiv.org/pdf/1704.05179.pdf
7.	bdhingra/quasar: Datasets for Question Answering by Search and Reading (github.com)
8.	nyu-dl/dl4ir-searchQA (github.com)
9.	https://drive.google.com/drive/u/2/folders/1kBkQGooNyG0h8waaOJpgdGtOnlb1S649
10.	https://github.com/thunlp/ERNIE
11.	Pretrained Encyclopedia: Weakly Supervised Knowledge-Pretrained Language Model | Papers With Code
12.	nyu-dl/dl4ir-searchQA (github.com)
13.	BERT base vs BERT large (opengenus.org)
14. https://paperswithcode.com/method/distillbert
15. https://arxiv.org/abs/1910.01108
16. https://medium.com/huggingface/distilbert-8cf3380435b5
17. https://awsdocs-neuron.readthedocs-hosted.com/en/latest/src/examples/tensorflow/huggingface_bert/huggingface_bert.html
18. https://huggingface.co/transformers/master/main_classes/pipelines.html
19. https://medium.com/analytics-vidhya/hugging-face-transformers-how-to-use-pipelines-10775aa3db7e
20. https://medium.com/microsoftazure/accelerate-your-nlp-pipelines-using-hugging-face-transformers-and-onnx-runtime-2443578f4333
21. https://theaidigest.in/zero-shot-classification-using-huggingface-transformers-pipeline/
22. https://stackoverflow.com/questions/64685243/getting-sentence-embedding-from-huggingface-feature-extraction-pipeline
23. https://www.holisticseo.digital/python-seo/nltk/lemmatize#:~:text=How%20to%20use%20NLTK%20Lemmatizaters%20with%20Part%20of,%E2%80%9CJ%E2%80%9D%2C%20%E2%80%9CV%E2%80%9D%20with%20%E2%80%9Cstartswith%E2%80%9D%20method.%20More%20items...%20
24. https://www.geeksforgeeks.org/python-lemmatization-with-nltk/
25. https://www.guru99.com/stemming-lemmatization-python-nltk.html
26. https://textminingonline.com/dive-into-nltk-part-iv-stemming-and-lemmatization
27. https://qa.fastforwardlabs.com/no%20answer/null%20threshold/bert/distilbert/exact%20match/f1/robust%20predictions/2020/06/09/Evaluating_BERT_on_SQuAD.html
28. https://docs.allennlp.org/models/main/models/rc/metrics/squad_em_and_f1/
29. https://stackoverflow.com/questions/52229059/em-score-in-squad-challenge
30. https://deepai.org/machine-learning-glossary-and-terms/f-score
31. https://vidyasheela.com/what-is-f1-score-and-what-is-its-importance-in-machine-learning/
32. https://www.myaccountingcourse.com/accounting-dictionary/f1-score
